In [9]:
import pandas as pd
import os

In [10]:
# Загружаем данные
cur_dir = os.getcwd()
netflix_data_dir = os.path.join(cur_dir, 'data')
netflix_data_path = os.path.join(netflix_data_dir, 'netflix_titles.csv')
df = pd.read_csv(netflix_data_path)
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [11]:
# Создаём файл sqlite
import sqlite3

connection = sqlite3.connect(os.path.join(cur_dir, 'db.db'))
connection.close()

In [12]:
# Работаем с актёрами
cast = df.loc[:, 'cast'].copy().dropna().str.split(',').apply(pd.Series).stack().str.strip().unique()
cast_series = pd.Series(cast, name='Actor')
with sqlite3.connect(os.path.join(cur_dir, 'db.db')) as connection:
    cast_series.to_sql('Actor', connection)
cast_series

0                   Ama Qamata
1                  Khosi Ngema
2                Gail Mabalane
3               Thabang Molaba
4             Dillon Windvogel
                 ...          
36434              Ryan Newman
36435          Raaghav Chanana
36436            Malkeet Rauni
36437           Anita Shabdish
36438    Chittaranjan Tripathy
Name: Actor, Length: 36439, dtype: object

ValueError: Table 'Actor' already exists.

In [ ]:
# Работаем с режисёрами
directors = df.loc[:, 'director'].copy().dropna().str.split(',').apply(pd.Series).stack().str.strip().unique()
directors_series = pd.Series(directors, name='Director')
with sqlite3.connect(os.path.join(cur_dir, 'db.db')) as connection:
    directors_series.to_sql('Director', connection)
directors_series

0               Kirsten Johnson
1               Julien Leclercq
2                 Mike Flanagan
3                 Robert Cullen
4                José Luis Ucha
                 ...           
4988                     Mu Chu
4989    Chandra Prakash Dwivedi
4990            Majid Al Ansari
4991               Peter Hewitt
4992                Mozez Singh
Name: Director, Length: 4993, dtype: object

In [ ]:
# Работаем со странами
countries = df.loc[:, 'country'].copy().dropna().str.split(',').apply(pd.Series).stack().str.strip().unique()
countries_series = pd.Series(countries, name='Country')
with sqlite3.connect(os.path.join(cur_dir, 'db.db')) as connection:
    countries_series.to_sql('Country', connection)
countries_series

0      United States
1       South Africa
2              India
3              Ghana
4       Burkina Faso
           ...      
118            Sudan
119           Panama
120           Uganda
121     East Germany
122       Montenegro
Name: Country, Length: 123, dtype: object

In [ ]:
# type
show_type = df.loc[:, 'type'].copy().dropna().str.split(',').apply(pd.Series).stack().str.strip().unique()
type_series = pd.Series(show_type, name='Actor')
with sqlite3.connect(os.path.join(cur_dir, 'db.db')) as connection:
    type_series.to_sql('Type', connection)
type_series

0      Movie
1    TV Show
Name: Actor, dtype: object

In [ ]:
show_data = df.loc[:, ['title', 'date_added', 'duration', 'release_year', 'rating', 'description']].copy()
with sqlite3.connect(os.path.join(cur_dir, 'db.db')) as connection:
    show_data.to_sql('Show', connection)
show_data

,title,date_added,duration,release_year,rating,description
0,Dick Johnson Is Dead,"September 25, 2021",90 min,2020,PG-13,"As her father nears the end of his life, filmm..."
1,Blood & Water,"September 24, 2021",2 Seasons,2021,TV-MA,"After crossing paths at a party, a Cape Town t..."
2,Ganglands,"September 24, 2021",1 Season,2021,TV-MA,To protect his family from a powerful drug lor...
3,Jailbirds New Orleans,"September 24, 2021",1 Season,2021,TV-MA,"Feuds, flirtations and toilet talk go down amo..."
4,Kota Factory,"September 24, 2021",2 Seasons,2021,TV-MA,In a city of coaching centers known to train I...
...,...,...,...,...,...,...
8802,Zodiac,"November 20, 2019",158 min,2007,R,"A political cartoonist, a crime reporter and a..."
8803,Zombie Dumb,"July 1, 2019",2 Seasons,2018,TV-Y7,"While living alone in a spooky town, a young g..."
8804,Zombieland,"November 1, 2019",88 min,2009,R,Looking to survive in a world taken over by zo...
8805,Zoom,"January 11, 2020",88 min,2006,PG,"Dragged from civilian life, a former superhero..."
